In [79]:
# Importujemy niezbędne biblioteki:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn import set_config
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier

In [2]:
DATA_PATH = "titanic.txt"

## Funkcje pomocnicze

# Krok 1: Wczytywanie danych

W tej sekcji wczytujemy dane z pliku `titanic.txt`, zamieniamy brakujące wartości oznaczone jako `"?"` na `NaN` i nadajemy kolumnom czytelne nazwy. Dodatkowo konwertujemy kolumny `age` i `fare` na typ liczbowy.

In [4]:
# Wczytanie danych
df = pd.read_csv(DATA_PATH)

# Zamiana "?" na NaN
df = df.replace("?", np.nan)

# Poprawa nazw kolumn
df.columns = [
    'pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch',
    'ticket', 'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'
]

# Zmiana na liczby
df['age'] = df['age'].astype(float)
df['fare'] = df['fare'].astype(float)

df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


# Krok 2: Eksploracja danych

## Cel
W tej sekcji przeprowadzamy **eksploracyjną analizę danych (EDA)**, aby lepiej poznać strukturę zbioru, typy kolumn, brakujące wartości oraz podstawowe statystyki opisowe.

## Znaczenie kolumn
- `pclass` – klasa biletu  
- `survived` – czy przeżył (0 – zginął, 1 – przeżył)  
- `name` – nazwisko i imię  
- `sex` – płeć  
- `age` – wiek  
- `sibsp` – liczba rodzeństwa / małżonków  
- `parch` – liczba rodziców / dzieci  
- `ticket` – numer biletu  
- `fare` – cena biletu  
- `cabin` – numer kabiny pasażera  
- `embarked` – port, z którego wypłynął pasażer  
- `boat` – numer łodzi ratunkowej  
- `body` – numer ciała  
- `home.dest` – miejsce zamieszkania


In [5]:
df.info()  # podstawowe informacje
df.describe(include='all')  # statystyki opisowe, także dla kolumn kategorycznych

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    object 
 13  home.dest  745 non-null    object 
dtypes: float64(2), int64(4), object(8)
memory usage: 143.3+ KB


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
count,1309.000000,1309.000000,1309,1309,1046.000000,1309.000000,1309.000000,1309,1308.000000,295,1307,486,121,745
unique,NaN,NaN,1307,2,NaN,NaN,NaN,929,NaN,186,3,27,121,369
top,NaN,NaN,"Connolly, Miss. Kate",male,NaN,NaN,NaN,CA. 2343,NaN,C23 C25 C27,S,13,135,"New York, NY"
freq,NaN,NaN,2,843,NaN,NaN,NaN,11,NaN,6,914,39,1,64
mean,2.294882,0.381971,NaN,NaN,29.881135,0.498854,0.385027,NaN,33.295479,NaN,NaN,NaN,NaN,NaN
std,0.837836,0.486055,NaN,NaN,14.413500,1.041658,0.865560,NaN,51.758668,NaN,NaN,NaN,NaN,NaN
min,1.000000,0.000000,NaN,NaN,0.166700,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,2.000000,0.000000,NaN,NaN,21.000000,0.000000,0.000000,NaN,7.895800,NaN,NaN,NaN,NaN,NaN
50%,3.000000,0.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN,NaN,NaN,NaN
75%,3.000000,1.000000,NaN,NaN,39.000000,1.000000,0.000000,NaN,31.275000,NaN,NaN,NaN,NaN,NaN


## Liczba unikalnych wartości w kolumnach

Tworzymy tabelę pokazującą dla każdej kolumny:
- liczbę unikalnych etykiet,
- subiektywną ocenę “mocy zbioru” (mała / średnia / duża),  
co pozwala lepiej zrozumieć charakter kolumn (kategoryczna vs ciągła, duplikaty, itp.).


In [6]:
pd.DataFrame({
    'Kolumna' : ['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
    'Ilość etykiet' : [
        len(df.pclass.unique()), len(df.survived.unique()), len(df.name.unique()), len(df.sex.unique()),
        len(df.age.unique()), len(df.sibsp.unique()), len(df.parch.unique()), len(df.ticket.unique()),
        len(df.fare.unique()), len(df.cabin.unique()), len(df.embarked.unique()),
        len(df.boat.unique()), len(df.body.unique()), len(df['home.dest'].unique())],
    'Moc zbioru' : ['mała', 'mała', 'duża', 'mała', 'duża', 'mała', 'mała', 'duża', 'duża', 'duża', 'mała', 'średnia', 'duża', 'duża']
})

,Kolumna,Ilość etykiet,Moc zbioru
0,pclass,3,mała
1,survived,2,mała
2,name,1307,duża
3,sex,2,mała
4,age,99,duża
5,sibsp,7,mała
6,parch,8,mała
7,ticket,929,duża
8,fare,282,duża
9,cabin,187,duża


In [7]:
print(f'Liczba pasażerów to {len(df)}')

Liczba pasażerów to 1309


# Krok 3: Redukcja zmiennych / przygotowanie cech

Niektóre kolumny mają dużą liczbę unikalnych wartości, co utrudnia analizę i modelowanie.  
Tworzymy nowe, uproszczone zmienne na potrzeby dalszej eksploracji i modelowania.  
Na przykład kolumna `cabin` jest redukowana do pierwszej litery (`cabin_reduced`), aby zmniejszyć liczbę kategorii.


In [8]:
df['cabin_reduced'] = df['cabin'].astype(object).str[0]
df['cabin_reduced'].unique()

array(['B', 'C', 'E', 'D', 'A', nan, 'T', 'F', 'G'], dtype=object)

## Wydzielenie tytułów z kolumny `name`

Z kolumny `name` wydzielamy tytuły pasażerów (np. Mr, Mrs, Miss), które mogą wpływać na przeżywalność.  
Następnie mapujemy tytuły na liczby, aby ułatwić dalsze modelowanie.


In [9]:
# Konwersja kolumny 'name' na string i wydzielenie tytułów (np. Mr, Mrs, Miss)
df['name'] = df['name'].astype(str)
df['titles'] = df['name'].str.extract(r',\s*([^\.]+)\.')

# Mapowanie tytułów na liczby
title_map = {
    'Miss': 0, 'Master': 1, 'Mr': 2, 'Mrs': 3, 'Col': 4, 'Mme': 5, 'Dr': 6,
    'Major': 7, 'Capt': 8, 'Lady': 9, 'Sir': 10, 'Mlle': 11, 'Dona': 12,
    'Jonkheer': 13, 'the Countess': 14, 'Don': 15, 'Rev': 16, 'Ms': 17
}
df['titles'] = df['titles'].map(title_map)


## Grupowanie wieku i obliczanie wielkości rodziny

- `age_group`: dzieli pasażerów na trzy grupy wiekowe (dzieci, dorośli, starsi).  
- `family`: liczba osób w rodzinie (rodzeństwo/małżonek + rodzice/dzieci + sam pasażer).


In [10]:
# Grupy wiekowe
df['age_group'] = pd.cut(
    df['age'],
    bins=[0, 15, 50, np.inf],
    labels=[0, 1, 2],
    right=True
)

# Wielkość rodziny
df['family'] = df['sibsp'] + df['parch'] + 1


## Kodowanie zmiennych kategorycznych

- `sex`: zamieniamy płeć na zmienną binarną (0 = mężczyzna, 1 = kobieta).  
- `embarked`: port wypłynięcia zamieniamy na liczby, aby ułatwić analizę i modelowanie.


In [11]:
# Kodowanie płci
df['sex'] = df['sex'].astype(str).map({'male': 0, 'female': 1})

# Kodowanie portu wypłynięcia
df['embarked'] = df['embarked'].map({'C': 0, 'Q': 1, 'S': 2})


## Redukcja kolumny `home.dest`

Tworzymy nową kolumnę `home.dest.reduced`, w której:
- Usuwamy początkową część adresu (przed przecinkiem), aby zachować tylko miasto lub kraj docelowy.
- Upraszczamy zapisy, usuwając fragmenty po znakach `/` i kolejnych przecinkach.

Celem jest zmniejszenie liczby unikalnych wartości w kolumnie i ułatwienie analizy geograficznej pasażerów.


In [12]:
df['home.dest.reduced'] = df['home.dest'].str.replace(r'^.*?,\s*', '', regex=True)
df['home.dest.reduced'] = df['home.dest.reduced'].str.replace(r'/.*?,\s*', '', regex=True)

## Normalizacja kolumny `home.dest`

- Redukujemy wartości w kolumnie `home.dest` do uproszczonych nazw krajów lub regionów (`home.dest.reduced`).  
- Dzielimy skomplikowane zapisy (np. z wieloma miastami lub stanami) i mapujemy je na pojedyncze kraje lub grupy, ułatwiając analizę geograficzną.  
- Tworzymy słownik `continent_map`, który przypisuje każdą unikalną wartość do odpowiadającego kraju lub regionu.


In [13]:
df['home.dest.reduced'] = df['home.dest.reduced'].astype(str).str.split('/').str[0].str.strip()
continent_map = {
    'MO': 'USA',
    'PQ ON': 'Canada',
    'NY': 'USA',
    'NI': 'UK',
    'Queens, NY': 'USA',
    'Uruguay': 'Uruguay',
    'France': 'France',
    'nan': 'nan',
    'Yorks': 'UK',
    'PQ': 'Canada',
    'MN': 'USA',
    'CA': 'USA',
    'MI': 'USA',
    'Sweden DC': 'Sweden',
    'NJ': 'USA',
    'OH': 'USA',
    'England Cleveland, Ohio': 'England\tUSA',
    'MB': 'Canada',
    'England Ohio': 'England\tUSA',
    'WA': 'USA',
    'NE': 'USA',
    'PA': 'USA',
    'CO': 'USA',
    'MA': 'USA',
    'DC': 'USA',
    'Philadelphia, PA': 'USA',
    'Berkshire NY': 'USA',
    'Staffs': 'UK',
    'ND': 'USA',
    'NY NY': 'USA',
    'England': 'England',
    'England / Belfast': 'England\tIreland',
    'BC': 'Canada',
    'IN': 'USA',
    'WI': 'USA',
    'Peru': 'Peru',
    'AB': 'Canada',
    'MN IA': 'USA',
    'London / Paris': 'UK\tFrance',
    'Essex': 'UK',
    'Switzerland': 'Switzerland',
    'Lancashire': 'UK',
    'France NY': 'France\tUSA',
    'CT': 'USA',
    'Surrey': 'UK',
    'London / Middlesex': 'UK',
    'London / Birmingham': 'UK',
    'IL': 'USA',
    'NY /  Stamford CT': 'USA',
    'France New York, NY': 'France\tUSA',
    'Liverpool': 'UK',
    'VT': 'USA',
    'London': 'UK',
    'OR': 'USA',
    'Sweden': 'Sweden',
    'RI': 'USA',
    'Cuba': 'Cuba',
    'Belfast': 'Ireland',
    'Spain': 'Spain',
    'ON': 'Canada',
    'Netherlands': 'Netherlands',
    'PA NY': 'USA',
    'NY / Greenwich CT': 'USA',
    'Long Island, NY': 'USA',
    'WV': 'USA',
    'London Hants': 'UK',
    'Ohio / ? Paris / New York': 'USA\tFrance\tUSA',
    'Mexico': 'Mexico',
    'Utah': 'USA',
    'ME': 'USA',
    'NY / Briarcliff Manor NY': 'USA',
    'Switzerland PA': 'Switzerland\tUSA',
    'NS': 'Canada',
    'NY DC': 'USA',
    'PA, USA': 'USA',
    'Argentina NJ': 'Argentina\tUSA',
    'England Houghton, MI': 'England\tUSA',
    'Cornwall OH': 'UK\tUSA',
    'Guernsey': 'UK',
    'Avon FL': 'USA',
    'Dorset MI': 'UK\tUSA',
    'FL': 'USA',
    'India MI': 'India\tUSA',
    'Cornwall MI': 'UK\tUSA',
    'NY NJ': 'USA',
    'Chavez County, NM': 'USA',
    'South Africa WA': 'South Africa\tUSA',
    'Sweden IL': 'Sweden\tUSA',
    'England CA': 'England\tUSA',
    'Hants': 'UK',
    'Thailand IL': 'Thailand\tUSA',
    'Huntingdonshire': 'UK',
    'Finland Ashtabula, Ohio': 'Finland\tUSA',
    'Hants ID': 'UK\tUSA',
    'Pennsylvania': 'USA',
    'UT': 'USA',
    'India Pittsburgh, PA': 'India\tUSA',
    'Brazil': 'Brazil',
    'Italy / California': 'Italy\tUSA',
    'Southampton': 'UK',
    'Spain Cuba': 'Spain\tCuba',
    'Norway Bayonne, NJ': 'Norway\tUSA',
    'India / Pennsylvania': 'India\tUSA',
    'Cornwall': 'UK',
    'WA OH': 'USA',
    'Surrey / Chicago': 'UK\tUSA',
    'Essex MB': 'UK\tCanada',
    'SD': 'USA',
    'London MB': 'UK\tCanada',
    'Finland MI': 'Finland\tUSA',
    'Japan': 'Japan',
    'Paris / Haiti': 'France\tHaiti',
    'Switzerland IA': 'Switzerland\tUSA',
    'Somerset': 'UK',
    'Paris': 'France',
    'SK': 'Canada',
    'Australia': 'Australia',
    'Middlesex': 'UK',
    'Denmark': 'Denmark',
    'NJ andOhio': 'USA',
    'KY': 'USA',
    'Glasgow': 'UK',
    'Germany': 'Germany',
    'Kent NY': 'UK\tUSA',
    'England NJ': 'England\tUSA',
    'England MT': 'England\tUSA',
    'Devon': 'UK',
    'Russia': 'Russia',
    'NH': 'USA',
    'Devon MI': 'UK\tUSA',
    'Sussex': 'UK',
    'England Ohio and Milwaukee, WI': 'England\tUSA',
    'DE': 'USA',
    'MT': 'USA',
    'Kent MI': 'UK\tUSA',
    'England / Massachusetts': 'England\tUSA',
    'Co Washington, VT': 'USA',
    'England CT': 'England\tUSA',
    'England PQ': 'England\tCanada',
    'England RI': 'England\tUSA',
    'England OR': 'England\tUSA',
    'Finland Hoboken, NJ': 'Finland\tUSA',
    'Sweden Brooklyn, NY': 'Sweden\tUSA',
    'England Norfolk, VA': 'England\tUSA',
    'VA': 'USA',
    'Finland Astoria, OR': 'Finland\tUSA',
    'Argentina': 'Argentina',
    'Middlesex or Erdington, Birmingham': 'UK',
    'England New York, NY': 'England\tUSA',
    'Norway': 'Norway',
    'Finland New York, NY': 'Finland\tUSA',
    'Sweden Ministee, MI': 'Sweden\tUSA',
    'Sweden Minneapolis, MN': 'Sweden\tUSA',
    'Sweden Bloomington, IL': 'Sweden\tUSA',
    'Sweden Moune, IL': 'Sweden\tUSA',
    'Finland New York': 'Finland\tUSA',
    'Syria': 'Syria',
    'Norway New York': 'Norway\tUSA',
    'Sweden New York': 'Sweden\tUSA',
    'Cardiff, Wales': 'UK',
    'Co Cork, Ireland Glens Falls, NY': 'Ireland\tUSA',
    'Sweden Chicago, IL': 'Sweden\tUSA',
    'Chelmsford, England': 'UK',
    'Ireland Roxbury, MA': 'Ireland\tUSA',
    'Co Cork, Ireland New York, NY': 'Ireland\tUSA',
    'Ireland Charlestown, MA': 'Ireland\tUSA',
    'Ireland New York, NY': 'Ireland\tUSA',
    'Croatia': 'Croatia',
    'Sweden Fower, MN': 'Sweden\tUSA',
    'Sweden Huntley, IL': 'Sweden\tUSA',
    'Ireland Hartford, CT': 'Ireland\tUSA',
    'Southampton, Hants': 'UK',
    'Greece': 'Greece',
    'Portugal': 'Portugal',
    'Ireland Sherbrooke, PQ': 'Ireland\tCanada',
    'Ireland': 'Ireland',
    'Austria': 'Austria',
    'England Cleveland, OH': 'England\tUSA',
    'England Newark, NJ': 'England\tUSA',
    'Austria-Hungary': 'Austria-Hungary',
    'IA': 'USA',
    'England Pontiac, MI': 'England\tUSA',
    'England Bedford, OH': 'England\tUSA',
    'England Wichita, KS': 'England\tUSA',
    'Co Sligo, Ireland New York, NY': 'Ireland\tUSA',
    'Sweden Joliet, IL': 'Sweden\tUSA',
    'Sweden Jerome Junction, AZ': 'Sweden\tUSA',
    'USA': 'USA',
    'Co Longford, Ireland New York, NY': 'Ireland\tUSA',
    'Sussex, England Essex Co, MA': 'UK\tUSA',
    'Somerset, England': 'UK',
    'Ireland Washington, DC': 'Ireland\tUSA',
    'Kent, England Detroit, MI': 'UK\tUSA',
    'England Niagara Falls, NY': 'England\tUSA',
    'Surrey, England': 'UK',
    'Norway Portland, ND': 'Norway\tUSA',
    'Chicago, IL': 'USA',
    'Sweden New York, NY': 'Sweden\tUSA',
    'Norway Cameron, WI': 'Norway\tUSA',
    'Belgium OH': 'Belgium\tUSA'
}


## Tworzenie kolumny `countries`

- Na podstawie zredukowanej kolumny `home.dest.reduced` przypisujemy


In [14]:
df['countries'] = df['home.dest.reduced'].map(continent_map).str.split('\t')

## Mapowanie krajów na kontynenty

- Tworzymy słownik `country_to_continent`, który przypisuje każdemu krajowi odpowiadający kontynent.  
- Kolumna `countries` (lista krajów) jest mapowana na kontynenty, a wynik łączymy w jeden string z rozdzieleniem `/` dla pasażerów mających wiele krajów.  
- Puste lub nieznane wartości zastępujemy `NaN`.  

Celem jest uzyskanie zmiennej `continent`, która ułatwia analizę geograficzną pasażerów w kontekście kontynentów.


In [15]:
country_to_continent = {
    'Switzerland': 'Europe',
    'UK': 'Europe',
    'England': 'Europe',
    'Germany': 'Europe',
    'France': 'Europe',
    'Finland': 'Europe',
    'Ireland': 'Europe',
    'Belgium': 'Europe',
    'Netherlands': 'Europe',
    'Austria': 'Europe',
    'Austria-Hungary': 'Europe',
    'Norway': 'Europe',
    'Sweden': 'Europe',
    'Denmark': 'Europe',
    'Russia': 'Europe',
    'Greece': 'Europe',
    
    'USA': 'America',
    'Canada': 'America',
    'Mexico': 'America',
    'Cuba': 'America',
    'Brazil': 'America',
    'Argentina': 'America',
    'Uruguay': 'America',
    'Peru': 'America',

    'South Africa': 'Africa',

    'India': 'Asia',
    'Japan': 'Asia',
    'Thailand': 'Asia',
    'Syria': 'Asia',
    'Croatia': 'Europe',
    'Spain': 'Europe',
    'Portugal': 'Europe',
    
    'Australia': 'Oceania',
    
    'nan': np.nan
}
df['continent'] = df['countries'].apply(
    lambda x: [country_to_continent.get(c.strip(), np.nan) for c in x] 
    if isinstance(x, list) else np.nan
)
df['continent'] = df['continent'].apply(
    lambda x: '/'.join([c for c in x if isinstance(c, str)]) if isinstance(x, list) and len(x) > 0 else np.nan
)
df['continent'] = df['continent'].replace({None: np.nan, '': np.nan})


In [16]:
value_counts = df['continent'].value_counts(dropna=False)
df_counts = value_counts.reset_index()
df_counts

,continent,count
0,NaN,568
1,America,453
2,Europe,151
3,Europe/America,120
4,Asia/America,8
5,Asia,4
6,Africa/America,3
7,America/America,1
8,Oceania,1


In [17]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,...,boat,body,home.dest,cabin_reduced,titles,age_group,family,home.dest.reduced,countries,continent
0,1,1,"Allen, Miss. Elisabeth Walton",1,29.0000,0,0,24160,211.3375,B5,...,2,NaN,"St Louis, MO",B,0,1,1,MO,[USA],America
1,1,1,"Allison, Master. Hudson Trevor",0,0.9167,1,2,113781,151.5500,C22 C26,...,11,NaN,"Montreal, PQ / Chesterville, ON",C,1,0,4,PQ ON,[Canada],America
2,1,0,"Allison, Miss. Helen Loraine",1,2.0000,1,2,113781,151.5500,C22 C26,...,NaN,NaN,"Montreal, PQ / Chesterville, ON",C,0,0,4,PQ ON,[Canada],America
3,1,0,"Allison, Mr. Hudson Joshua Creighton",0,30.0000,1,2,113781,151.5500,C22 C26,...,NaN,135,"Montreal, PQ / Chesterville, ON",C,2,1,4,PQ ON,[Canada],America
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",1,25.0000,1,2,113781,151.5500,C22 C26,...,NaN,NaN,"Montreal, PQ / Chesterville, ON",C,3,1,4,PQ ON,[Canada],America


# Krok 4: Imputacja danych

- Niektóre kolumny w zbiorze nadal mają brakujące wartości.  
- W tym kroku wypełniamy braki, stosując różne strategie 
- Dzięki temu uzyskujemy kompletne dane, gotowe do dalszej analizy i modelowania.


In [18]:
# Imputacja countries według grup (titles, pclass, embarked, family, survived)
titles = df['titles'].unique()
pclasses = df['pclass'].unique()
ports = df['embarked'].unique()
families = df['family'].unique()
survived_vals = df['survived'].unique()

for title in titles:
    for pclass in pclasses:
        for port in ports:
            for fam in families:
                for surv in survived_vals:
                    group_idx = (
                        (df['titles'] == title) &
                        (df['pclass'] == pclass) &
                        (df['embarked'] == port) &
                        (df['family'] == fam) &
                        (df['survived'] == surv)
                    )
                    # Wyznaczamy modę tylko jeśli są nie-NaN wartości
                    mode_val = df.loc[group_idx & df['countries'].notna(), 'countries'].mode()
                    if not mode_val.empty:
                        # Wypełniamy brakujące wartości w tej grupie
                        mask = group_idx & df['countries'].isna()
                        if mask.any():  # upewniamy się, że istnieją brakujące
                            df.loc[mask, 'countries'] = mode_val.iloc[0]

# Na końcu globalna moda dla pozostałych braków
df.loc[df['countries'].isna(), 'countries'] = df['countries'].mode().iloc[0]

In [19]:
df = df.dropna(subset=['fare', 'embarked'])

In [20]:
for title in df['titles'].unique():
    for sib in df['sibsp'].unique():
        for pclass in df['pclass'].unique():
            for port in df['embarked'].unique():
                for surv in df['survived'].unique():
                    # Wyznaczamy modę w grupie
                    mode_val = df.loc[
                        (df['titles'] == title) &
                        (df['sibsp'] == sib) &
                        (df['pclass'] == pclass) &
                        (df['embarked'] == port) &
                        (df['survived'] == surv) &
                        (df['cabin_reduced'].notna()),
                        'cabin_reduced'
                    ].mode()
                    
                    if len(mode_val) > 0:
                        # Wypełniamy brakujące wartości modą grupy
                        df.loc[
                            (df['titles'] == title) &
                            (df['sibsp'] == sib) &
                            (df['pclass'] == pclass) &
                            (df['embarked'] == port) &
                            (df['survived'] == surv) &
                            (df['cabin_reduced'].isna()),
                            'cabin_reduced'
                        ] = mode_val[0]

# Na końcu wypełniamy resztę braków globalną modą
df.loc[:, 'cabin_reduced'] = df['cabin_reduced'].fillna(df['cabin_reduced'].mode()[0])

In [21]:
conditions = [
    df['survived'] == 1,                         # przeżył
    (df['survived'] == 0) & (df['body'].notna()), # zmarł, znamy numer ciała
    (df['survived'] == 0) & (df['body'].isna())   # zmarł, nie znamy numeru ciała
]

# Wartości odpowiadające warunkom
values = [0, 1, 2]

# Tworzymy nową kolumnę
df.loc[:, 'body'] = np.select(conditions, values)

In [22]:
df.loc[:, 'boat'] = np.where(df['boat'].notna(), 1, 0)

In [23]:
# Imputacja age_group według grup (pclass, family, embarked, survived)
for pclass in df['pclass'].unique():
    for fam in df['family'].unique():
        for port in df['embarked'].unique():
            for surv in df['survived'].unique():
                # Wyznaczamy modę w grupie
                mask_group = (
                    (df['pclass'] == pclass) &
                    (df['family'] == fam) &
                    (df['embarked'] == port) &
                    (df['survived'] == surv) &
                    (df['age_group'].notna())
                )
                mode_val = df.loc[mask_group, 'age_group'].mode()
                
                if not mode_val.empty:
                    # Wypełniamy brakujące wartości modą grupy
                    mask_na = (
                        (df['pclass'] == pclass) &
                        (df['family'] == fam) &
                        (df['embarked'] == port) &
                        (df['survived'] == surv) &
                        (df['age_group'].isna())
                    )
                    df.loc[mask_na, 'age_group'] = mode_val.iat[0]

# Na końcu globalna moda dla pozostałych braków
df.loc[:, 'age_group'] = df['age_group'].fillna(df['age_group'].mode().iat[0])


# Krok 5: Wybór kolumn do predykcji / analizy

- Wybieramy tylko te kolumny, które są istotne do dalszej analizy i modelowania.  
- Tworzymy nowy DataFrame `df_selected`, zawierający wybrane zmienne:  
  - cechy pasażera (`pclass`, `sex`, `age_group`, `family`, `titles`, `cabin_reduced`, `continent`)  
  - informacje o podróży (`sibsp`, `parch`, `embarked`, `boat`, `body`)  
  - kolumnę docelową (`survived`).  
- Dzięki temu łatwiej jest skupić się na istotnych danych i przygotować je do modelowania.


In [24]:
selected_cols = [
    'pclass', 'survived', 'sex', 'age_group', 'sibsp', 'parch',
    'embarked', 'boat', 'body', 'cabin_reduced',
    'titles', 'family', 'continent'
]

df_selected = df[selected_cols].copy()
df_selected.head()

,pclass,survived,sex,age_group,sibsp,parch,embarked,boat,body,cabin_reduced,titles,family,continent
0,1,1,1,1,0,0,2.0,1,0,B,0,1,America
1,1,1,0,0,1,2,2.0,1,0,C,1,4,America
2,1,0,1,0,1,2,2.0,0,2,C,0,4,America
3,1,0,0,1,1,2,2.0,0,1,C,2,4,America
4,1,0,1,1,1,2,2.0,0,2,C,3,4,America


In [34]:
cat_features = ['continent', 'cabin_reduced', 'embarked']

# Pipeline dla kolumn kategorycznych
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline([
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_features)
    ]
)

In [37]:
model = Pipeline([
    ('prep', preprocessor),
    ('clf', RandomForestClassifier(random_state=42))
])

In [49]:
X = df_selected.drop('survived', axis=1)
y = df_selected['survived']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model.fit(X_train, y_train)
acc = model.score(X_test, y_test)

print(f'Dokładność: {acc:.3f}')

Dokładność: 0.771


In [61]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83       162
           1       0.79      0.54      0.64       100

    accuracy                           0.77       262
   macro avg       0.78      0.73      0.74       262
weighted avg       0.77      0.77      0.76       262



In [42]:
set_config(display='diagram')

model

,steps,"[('prep', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [63]:
X_1 = df_selected.drop(['survived', 'parch', 'sibsp', 'boat', 'body'], axis=1)
X_1train, X_1test, y_1train, y_1test = train_test_split(
    X_1, y, test_size=0.2, random_state=42, stratify=y
)
model.fit(X_1train, y_1train)
acc = model.score(X_1test, y_1test)

print(f'Dokładność: {acc:.3f}')

Dokładność: 0.771


In [64]:
y_1pred = model.predict(X_1test)
print(classification_report(y_1test, y_1pred))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83       162
           1       0.79      0.54      0.64       100

    accuracy                           0.77       262
   macro avg       0.78      0.73      0.74       262
weighted avg       0.77      0.77      0.76       262



In [53]:
df_selected

,pclass,survived,sex,age_group,sibsp,parch,embarked,boat,body,cabin_reduced,titles,family,continent
0,1,1,1,1,0,0,2.0,1,0,1,0,1,1
1,1,1,0,0,1,2,2.0,1,0,2,1,4,1
2,1,0,1,0,1,2,2.0,0,2,2,0,4,1
3,1,0,0,1,1,2,2.0,0,1,2,2,4,1
4,1,0,1,1,1,2,2.0,0,2,2,3,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,1,0,1,0,0.0,0,1,5,0,2,8
1305,3,0,1,1,1,0,0.0,0,2,5,0,2,8
1306,3,0,0,1,0,0,0.0,0,1,5,2,1,8
1307,3,0,0,1,0,0,0.0,0,2,5,2,1,8


In [65]:
X_1 = df_selected.drop(['survived', 'parch', 'sibsp', 'boat', 'body', 'titles', 'age_group', 'pclass','family', 'sex'], axis=1)
X_1train, X_1test, y_1train, y_1test = train_test_split(
    X_1, y, test_size=0.2, random_state=42, stratify=y
)
model.fit(X_1train, y_1train)
acc = model.score(X_1test, y_1test)

print(f'Dokładność: {acc:.3f}')

Dokładność: 0.771


In [66]:
y_1pred = model.predict(X_1test)
print(classification_report(y_1test, y_1pred))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83       162
           1       0.79      0.54      0.64       100

    accuracy                           0.77       262
   macro avg       0.78      0.73      0.74       262
weighted avg       0.77      0.77      0.76       262



In [70]:
cat_features = ['continent', 'cabin_reduced', 'embarked']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ],
    remainder='passthrough'
)

# pipeline z Logistic Regression
model = Pipeline([
    ('prep', preprocessor),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

# podział danych
X = df_selected.drop(['survived', 'parch', 'sibsp', 'boat', 'body', 'titles', 'age_group', 'pclass','family', 'sex'], axis=1)
y = df_selected['survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# dopasowanie modelu
model.fit(X_train, y_train)

# ocena
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
acc = model.score(X_1test, y_1test)

print(f'Dokładność: {acc:.3f}')

              precision    recall  f1-score   support

           0       0.75      0.93      0.83       162
           1       0.81      0.51      0.63       100

    accuracy                           0.77       262
   macro avg       0.78      0.72      0.73       262
weighted avg       0.78      0.77      0.75       262

Dokładność: 0.767


In [73]:
cat_features = ['continent', 'cabin_reduced', 'embarked']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ],
    remainder='passthrough'
)

# pipeline z Logistic Regression
model = Pipeline([
    ('prep', preprocessor),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])
X = df_selected.drop(['survived', 'parch', 'sibsp', 'boat', 'body'], axis=1)
y = df_selected['survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# dopasowanie modelu
model.fit(X_train, y_train)

# ocena
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
acc = model.score(X_test, y_test)

print(f'Dokładność: {acc:.3f}')

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       162
           1       0.81      0.75      0.78       100

    accuracy                           0.84       262
   macro avg       0.83      0.82      0.82       262
weighted avg       0.83      0.84      0.83       262

Dokładność: 0.836


In [76]:
X.columns

Index(['pclass', 'sex', 'age_group', 'embarked', 'cabin_reduced', 'titles',
       'family', 'continent'],
      dtype='object')

In [80]:
# kolumny kategoryczne
cat_features = ['continent', 'cabin_reduced', 'embarked']

# preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ],
    remainder='passthrough'
)

# pipeline z HistGradientBoosting
model = Pipeline([
    ('prep', preprocessor),
    ('clf', HistGradientBoostingClassifier(
        max_iter=500,
        max_depth=4,
        learning_rate=0.05,
        random_state=42
    ))
])

# dane
X = df_selected.drop(['survived', 'parch', 'sibsp', 'boat', 'body'], axis=1)
y = df_selected['survived']

# podział train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# dopasowanie
model.fit(X_train, y_train)

# predykcja i ocena
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# accuracy
acc = model.score(X_test, y_test)
print(f'Dokładność: {acc:.3f}')

              precision    recall  f1-score   support

           0       0.88      0.95      0.91       162
           1       0.91      0.78      0.84       100

    accuracy                           0.89       262
   macro avg       0.89      0.87      0.87       262
weighted avg       0.89      0.89      0.88       262

Dokładność: 0.885
